# Processing excel tables

The aim of my work is to create the automated validation mechanism for the hospital reports from the excel sheets in the required format

In [1]:
import pandas as pd

In [2]:
def find_nums(df1, df2, to_populate):
    """Identifier function to validate names in the reports"""
    
    df = pd.DataFrame(index=to_populate, columns=['Both', 'Not in 1st', "Not in 2nd"])
    for i in to_populate:
        to_insert = [0,0,0]
        
        rep1_names = list(df1[df1['Медицинская организация'].str.lower() == i.lower()]['ФИО'])
        rep2_names = list(df2[df2['Юридическое лицо2'].str.lower() == i.lower()]["ФИО"])
        
        check_1 = tuple(rep1_names)

        for name in check_1:
            if name in rep2_names:
                to_insert[0] += 1
                rep1_names.pop(rep1_names.index(name))
                rep2_names.pop(rep2_names.index(name))
            else:
                to_insert[2] += 1
                rep1_names.pop(rep1_names.index(name))

        to_insert[1] += len(rep2_names)
        
        
        df.loc[i] = to_insert
    return df

In [3]:
path = 'Hospital_data.xlsx'

def solution(path):
    rep1 = pd.read_excel(path, sheet_name = "Отчет 1")
    rep2 = pd.read_excel(path, sheet_name = "Отчет 2")
    ref = pd.read_excel(path, sheet_name = "Справочник")

    mapping1 = dict(zip(ref['Отчет 1'], ref["С расшифровкой для свода"]))
    to_populate = pd.read_excel(path, sheet_name = "Получить свод")
    to_populate = to_populate = to_populate.iloc[6:, 0]
    to_populate.index.rename('entity')

    rep1 = rep1.iloc[:-1, [1,3]]
    rep1["Медицинская организация"] = rep1["Медицинская организация"].map(mapping1).fillna(rep1["Медицинская организация"])
    rep1 = rep1[rep1['Медицинская организация'].isin(to_populate)]
    rep1_patients = rep1.groupby('Медицинская организация')["ФИО"].count()

    df = pd.DataFrame(data=rep1_patients, index=to_populate)

    df.rename(columns={'ФИО': 'report_1'}, inplace=True)

    df.loc['ФГБУ ФКЦ ВМТ ФМБА РОССИИ', 'report_1'] = df.loc['ФГБУ ФКЦ ВМТ ФМБА России', 'report_1']

    rep1['ФИО'] = rep1['ФИО'].apply(lambda x: str(x).title())

    rep2['ФИО'] = rep2[["Фамилия", "Имя", "Отчество"]].astype(str).applymap(lambda x: x.title()).agg(" ".join, axis=1)
    rep2 = rep2.drop(["Фамилия", "Имя", "Отчество"], axis=1)

    rep2["Юридическое лицо2"] = rep2['Юридическое лицо'].replace({'ФГБУ "ФЕДЕРАЛЬНЫЙ КЛИНИЧЕСКИЙ ЦЕНТР ВЫСОКИХ МЕДИЦИНСКИХ ТЕХНОЛОГИЙ ФЕДЕРАЛЬНОГО МЕДИКО-БИОЛОГИЧЕСКОГО АГЕНТСТВА"': "ФГБУ ФКЦ ВМТ ФМБА России",
                                                                  "Королевская": "Королёвская", 'ГБУЗ МО Красногорская городская больница №': 'ГБУЗ МО Красногорская ГБ № ', 'ГБУЗ МОСКОВСКОЙ ОБЛАСТИ  "ПОДОЛЬСКАЯ ГОРОДСКАЯ КЛИНИЧЕСКАЯ БОЛЬНИЦА"': "ГБУЗ МО ПОДОЛЬСКАЯ ГКБ", "(?i)московская областная больница": "МОБ",'"': '', "имени": "им.",
                                                                  "областная клиническая больница": "ОКБ", "областная больница": "ОБ", "Московский областной научно-исследовательский клинический институт": "МОНИКИ",
                                                                  "Московская областная больница": "МОБ","первая районная больница": "ПРБ", "центральная районная клиническая больница":"ЦРКБ",
                                                                  "районная клиническая больница": "РКБ","МОСКОВСКОЙ ОБЛАСТИ": "МО", "детская городская больница": "ДГБ",
                                                                  "(?i)городская клиническая больница": "ГКБ","центральная районная больница": "ЦРБ", "(?i)районная больница": "РБ",
                                                                  "(?i)центральная городская больница": "ЦГБ", "(?i)ГОРОДСКАЯ БОЛЬНИЦА": "ГБ"}, regex=True)

    rep2 = rep2[rep2["Юридическое лицо2"].str.lower().isin(to_populate.str.lower())]
    rep2_patients = rep2.groupby(["Юридическое лицо2"])['ФИО'].count()

    df = df.merge(rep2_patients, left_on=df.index.str.lower(), right_on= rep2_patients.index.str.lower(), how='left').set_index(to_populate)
    df.drop('key_0', axis=1, inplace=True)
    df.rename(columns={'ФИО': 'report_2'}, inplace=True)

    df['DIFF'] = df.loc[:, 'report_2'].fillna(0) - df.loc[:, 'report_1'].fillna(0)

    diff_names = find_nums(rep1, rep2, to_populate)

    final = df.merge(diff_names, left_index=True, right_index=True)

    final['CHECK'] = final['DIFF'] - (final['Not in 1st'] - final['Not in 2nd'])
    a =pd.concat((pd.DataFrame(final.sum(axis=0).to_dict(), index=['Всего']), final))

    a.rename(columns = {'report_1': 'Пациенты на лечении по данным отчета 1', 'report_2': 'Пациенты на лечении по данным отчета 2',
                        'DIFF': 'Разница отчета 1 и отчета 2', 'Both': 'Есть в Отчете 1 и в Отчете 2 (совпадения)',
                        'Not in 1st': 'Есть в Отчете 2, нет в Отчете 1', 'Not in 2nd': 'Есть в отчете 1, нет в Отчете 2',
                        'CHECK': 'Проверка разницы'}, inplace=True)
    a.index.name = 'Наименование ЛПУ'
    a.to_excel("output.xlsx", na_rep='#N/A')
    
    return a


In [4]:
pd.set_option('display.max_rows', 70)

In [5]:
solution(path=path)

,Пациенты на лечении по данным отчета 1,Пациенты на лечении по данным отчета 2,Разница отчета 1 и отчета 2,Есть в Отчете 1 и в Отчете 2 (совпадения),"Есть в Отчете 2, нет в Отчете 1","Есть в отчете 1, нет в Отчете 2",Проверка разницы
Наименование ЛПУ,,,,,,,
Всего,5528.0,6250.0,722.0,3931,2319,1597,0.0
ГАУЗ МО КЦВМиР,409.0,NaN,-409.0,0,0,409,0.0
ГАУЗ МО ХИМКИНСКАЯ ОБ,573.0,613.0,40.0,504,109,69,0.0
ГБУЗ МО БАЛАШИХИНСКАЯ ОБ,172.0,214.0,42.0,156,58,16,0.0
ГБУЗ МО ВИДНОВСКАЯ РКБ,431.0,417.0,-14.0,358,59,73,0.0
ГБУЗ МО ДАВЫДОВСКАЯ РБ,56.0,52.0,-4.0,49,3,7,0.0
ГБУЗ МО ДМИТРОВСКАЯ ОБ,67.0,93.0,26.0,60,33,7,0.0
ГБУЗ МО ДОМОДЕДОВСКАЯ ЦГБ,90.0,107.0,17.0,73,34,17,0.0
ГБУЗ МО ЕГОРЬЕВСКАЯ ЦРБ,94.0,150.0,56.0,84,66,10,0.0


    1. Для Отчета 1: Отсутствующие значения для медицинских учреждений обусловлены в большинстве случаев отсутствием данных по этим местам в самом отчете. Однако, некоторые больницы например (Клинская ГБ) имеет неправильное определение в справочнике ("Клинская ОБ"). Также в своде присутствует дубликат: "ФГБУ ФКЦ ВМТ ФМБА РОССИИ" и "ФГБУ ФКЦ ВМТ ФМБА России". Так как это содержиться в форме отчета, я оставил оба этих названия и данные этих двух заведений полностью копируют друг друга.

    2. Для Отчета 2: Определения в справочнике не подходят ни под одно значение лечебных учреждений в отчете! Я вручную подогнал названия ЛПУ с требуемыми именами в своде. Все отсутствующие значения говорят об отсутствии данных в отчете.

    3. Я специально использовал функцию, которая будет считать пациентов с одинаковым ФИО как двух разных пациентов, так как не указано обратного. Таким образом в проверке нет несоответствий. В инном случае, погрешность составляла от 1 до 3 пациентов (С одинаковым ФИО).

    4. Функция экспортирует получившуюся таблицу в xlsx формате